In [1]:
import ee
import geemap
import os

In [2]:
ee.Initialize()

In [3]:
list_country = [
    'BR', #brésil
    'CO', #colombie
    'PE', #peru
    'CM', #cameroun
    'CF', #congo
    'CG', #rdc
    'GB', #gabon
    'UG', #Ouganda
    'GH', #ghana
    'IV', #côte d'Ivoire
    'ID', #indonésie
    'MY', #malaisie
]

In [4]:
Map = geemap.Map()
#Map

In [5]:
#add the AOI 
country_code = list_country[6] #gabon
country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', country_code))
Map.addLayer(country, {}, 'country_code')
Map.centerObject(country)

In [6]:
#add dataset 
dataset = ee.ImageCollection('NASA/MEASURES/GFCC/TC/v3').filter(ee.Filter.date('2010-01-01', '2010-12-31'))
treeCanopyCover = dataset.select('tree_canopy_cover')
treeCanopyCoverVis = {
  'min': 0.0,
  'max': 100.0,
  'palette': ['ffffff', 'afce56', '5f9c00', '0e6a00', '003800'],
}

country_gfcc_2010 = treeCanopyCover.mean().clip(country)

Map.addLayer(country_gfcc_2010, treeCanopyCoverVis, 'Tree Canopy Cover');

In [7]:
#load the ecozones 
gez_2010 = ee.Image('users/bornToBeAlive/gez_2010_wgs84')
country_gez_2010 =  gez_2010.select('b1').clip(country)

vizParam = {
    'min': 0, 
    'max': 21,
    'bands': ['b1'],
    'palette': ['black', 'green', 'blue'] 
}

Map.addLayer(country_gez_2010, vizParam, 'gez_2010_raster')

In [8]:
cdl = country_gez_2010.select('b1')

##mask everything that is non 0
masked = ee.Image(cdl).updateMask(cdl.eq(0))
country_gez_2010 = ee.Image(cdl).updateMask(cdl.neq(0))

#Map.addLayer(filtered)
Map.addLayer(masked, {'palette': 'red'}, 'masked_0')

In [9]:
#Get the list of values from the mosaic image
freqHist = country_gez_2010.reduceRegions(**{
    'reducer': ee.Reducer.frequencyHistogram(), 
    'collection': country, 
    'scale': 100, 
})

def getVal(feat):
    feat = ee.Feature(feat)
    vals = ee.Dictionary(feat.get('histogram')).keys()
    return ee.Feature(None, {'vals': vals})

#rewrite the dictionary
values = ee.List(freqHist.map(getVal).aggregate_array('vals')).flatten().distinct()

#print(values.getInfo())


In [10]:
#Run reduceToVectors per class by masking all other classes.
classes = country_gez_2010.reduceToVectors(**{
    'reducer': ee.Reducer.countEvery(), 
    'geometry': country, 
    'scale': 100,
    'maxPixels': 1e10
})
country_gez_2010_vector = ee.FeatureCollection(classes);

#Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte()

#Paint all the polygon edges with the same number and width, display.
outline = empty.paint(**{
    'featureCollection': country_gez_2010_vector,
    'color': 1,
    'width': 3
})
    
Map.addLayer(outline, {'palette': '000000'}, 'gez_2010_vector')

In [11]:
#add masking input
out_dir = os.path.join(os.path.expanduser('~'), 'downloads')
raw_stats = os.path.join(out_dir, country_code+'_raw.csv') 

geemap.zonal_statistics(country_gfcc_2010, country_gez_2010_vector, raw_stats, statistics_type='FIXED_HIST', hist_min=0, hist_max=100, hist_steps=50, scale=100)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /home/prambaud/downloads/GB_raw.csv


In [ ]:
#create the list of zones
#as ther are no names in the tiff file 
list_zones = {
41: 'Boreal coniferous forest',
43: 'Boreal mountain system',
42: 'Boreal tundra woodland',
50: 'Polar',
24: 'Subtropical desert',
22: 'Subtropical dry forest',
21: 'Subtropical humid forest',
25: 'Subtropical mountain system',
23: 'Subtropical steppe',
32: 'Temperate continental forest',
34: 'Temperate desert',
35: 'Temperate mountain system',
31: 'Temperate oceanic forest',
33: 'Temperate steppe',
15: 'Tropical desert',
13: 'Tropical dry forest',
12: 'Tropical moist forest',
16: 'Tropical mountain system',
11: 'Tropical rainforest',
14: 'Tropical shrubland',
90: 'Water'
}
#list_zones

In [ ]:
##need first to see how it looks like

import numpy as np
import csv
import pandas as pd
from bqplot import *

data = pd.read_csv(raw_stats, sep=',')

In [ ]:
#enlever les lignes Nan (vérifier dans le tableau d'avant qu'elles ne sont pas trop grandes)
data = data.dropna()
#data

In [ ]:
#recuperer les valeurs de label
ecozones = data.label.unique()
#ecozones

In [ ]:
#aggreger les lignes avec les même valeurs 
dummy = []
for i in range(0, 100, 2):
        dummy.append("{:.1f}".format(i))
stats = pd.DataFrame(dummy, columns=['treecover'])

#stats =[]
for index, ecozone in enumerate(ecozones): 
    patches = data.loc[data.label == ecozone]
    label = []
    for i in range(0, 100, 2):
        label.append(["{:.1f}".format(i), 0])
        
    for index, row in patches.iterrows():
        tmp = json.loads(row['histogram'])        
        for index, value in enumerate(tmp):
            label[index][1] += value[1]
            
    label = pd.DataFrame(label, columns=['treecover', list_zones[ecozone]])
    stats = pd.merge(left=stats, right=label, left_on='treecover', right_on='treecover')
    
#l'écrire dans un fichier
out_stats = os.path.join(out_dir, country_code+'_stats.csv') 
stats.to_csv(out_stats, index=False) 

In [ ]:
import bqplot as bq

x_sc = bq.OrdinalScale()
ax_x = bq.Axis(label='treecover', scale=x_sc)


x= []
for i in range(0, 100, 2):
    x.append(i)

figs = []
for ecozone in ecozones:
    y_sc = bq.LinearScale(max=stats[list_zones[ecozone]].max())
    ax_y = bq.Axis(label='surface (px)', scale=y_sc, orientation='vertical')
    y = []
    for index, row in stats.iterrows():
        y.append(row[list_zones[ecozone]])
    
    mark = bq.Bars(x=x, y=y, scales={'x': x_sc, 'y': y_sc})
    
    fig_hist = bq.Figure(
        marks=[mark], 
        axes=[ax_x, ax_y], 
        padding_x=0.025, 
        padding_y=0.025
    )
    
    fig_hist.layout.width = 'auto'
    fig_hist.layout.height = 'auto'
    fig_hist.layout.min_width = "300px"
    fig_hist.layout.min_height = '300px'
    
    figs.append(fig_hist)
    
for index, fig in enumerate(figs):
    fig.title = list_zones[ecozones[index]]
    display(fig)